This is a 72-hour challenge I did in late March 2022 for a data scientist position at an Indonesian multicompany, specifically at their media daughter company. The premise of the challenge is to recommend three to five apps from Google Playstore based on number of downloads, rating, and positive reviews. The constraint is the app must not be overly popular (e.g. Instagram, WhatsApp, Facebook).

<h1><center>Five Recommended Apps from Google Playstore</center></h1>

## INTRODUCTION

I believe that recommendation should be given based on outstading performance, and that includes apps. However, that may be difficult for newly launched apps with small number of users as the app lacks exposure towards wider audience and may underperform against Application Search Optimization (ASO). I decided to take up the task of recommending users of new apps worth trying using sentiment analysis in Python on thousands of reviews from 2010 to 2018, and write an article about it.

The purpose of the task is to mutually benefit users and app developers. I hope to appeal to users through my data-backed recommendation that there are apps which are as functional as the more popular apps out there. With increased number of users, these apps will have more organic data for further development and generate revenue.

My recommendation will be based on a few KPI's of an app such as number of installs, reasonable number of positive reviews and average rating in the last thirty days.

##  DATA CLEANING

Detecting and correcting (or removing) corrupt or inaccurate records from a record set, table, or database and refers to identifying incomplete, incorrect, inaccurate or irrelevant parts of the data and then replacing, modifying, or deleting the dirty or coarse data.

First, I need to import the required packages and dataset. In this project, I will be using `Numpy` for ... and `Pandas` for data processing. I will also be using two datasets, `app.csv` and `reviews.csv`.

In [1]:
# importing pacakges
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# initialize view settings
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', True)

# importing dataset    
app = pd.read_csv('/kaggle/input/provided-data/playstore_apps.csv')
reviews = pd.read_csv('/kaggle/input/provided-data/app_reviews.csv')

`app` contains inf

In [2]:
app.head(3)

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up


In [3]:
app.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


## DATA EXPLORATION

Initial data analysis, whereby a data analyst uses visual exploration to understand what is in a dataset and the characteristics of the data, rather than through traditional data management systems.

## FEATURE ENGINEERING

Using domain knowledge to extract features from raw data.

## SENTIMENT ANALYSIS

Using natural language processing, text analysis, computational linguistics, and biometrics to systematically identify, extract, quantify, and study affective states and subjective information. In this case, I will perform "simple" sentiment analysis.

## CONCLUSION

## DOCUMENTATION